# Paillier's encryption Scheme

The Paillier crypto system, invented by and named after Pascal Paillier in 1999, is a probabilistic asymmetric algorithm for public key cryptography. The problem of computing $n-th$ residue classes is believed to be computationally difficult. The decisional composite residuosity assumption is the intractability hypothesis upon which this cryptosystem is based. 

$$
\alpha +_E \beta = E(\alpha + \beta \ mod\ N)\\
x\times_E \alpha = E(x\alpha \ mod \ N)\\
\oplus_{i=1}^{t+1}a_i = a_1 +_E a_2 +_E \cdots +_E a_{t+1}
$$


### Key Generation:

1. Choose two large prime numbers $P$ and $Q$ randomly and indenpendently of each other such that $gcd(PQ, (P-1)(Q-1))=1$. This property is assured if both primes are qual lenth.

In [34]:
from klefki.const import (SECP256K1_P as P, SECP256K1_N as Q)

In [35]:
from math import gcd

assert gcd(P * Q, (P - 1) * (Q - 1)) == 1

2. Compute $N=PQ$ and $\lambda(N)=lcm(P-1, Q-1)$ be the Carmichael function of N.

In [36]:
from klefki.numbers import lcm

In [44]:
N = P * Q
Lam = lcm(P - 1, Q - 1)

3. Select random integer $\Gamma$ where $\Gamma \in \mathbf{Z}_{n^2}^*$.

In [45]:
from random import randint

G = randint(0, N ** 2)

4. Ensure $n$ devides the order of $g$ by checking the existence of the following modular multiplicative inverse: 

$$
\mu = (L(G^{\lambda(N)}\mod N^2))^{-1} \mod \ N
$$

Where $L$ be a function defined over the set $\{x\in \mathbf{Z}_{N^2}: x=1 \ mod \ N\}$ computed as

$$
L(x) = \frac{x-1}{N}
$$

In [47]:
from klefki.numbers import invmod

L = lambda x: (x - 1) // N
M = invmod(L(pow(G, Lam, N**2)), N)

The public key is $(N, \Gamma)$ and the secret key is $(\lambda(N), \mu)$

If using p,q of equivalent length, a simpler variant of the above key generation steps would be to set $g = n + 1 , λ = φ ( n )$, and $\mu = φ(n)^{-1}\ mod \ n$, where $φ(n)=(p-1)(q-1)$

In [48]:
print("PrivKey: %s, %s" % (Lam, M))

PrivKey: 2234634654990432849929004166367641021238215826767191291571707378398254532167475902984296517123225539202576657105730699764493290075143829571044458305451072, 12200147506290979449946411576842643915916356420380452765626501351126394208948833340910730704106417571576060982965207425176265757732798767761726178986379169


In [49]:
print("PubKey: %s; %s" % (N, G))

PubKey: 13407807929942597099574024998205846127429294960603147749430244270389527193005087002084253735130200377185477318450090306135904455919285040173416176828872431; 162139919707494919108141824856419713113334801225661567291106387446214183502085296253143434026196824563624278347752328822939233661085480031623983085474857216687960877726268525548563765838477827636326324736023541151721315347268552448839487433431050987219353087547486697936520602517411783477304103163985310480133


### Encryption:

To Encrypt a message $m \in Z_N$, select $x \in_R Z_N^*$ and return $c=\Gamma^mx^N\ mod \ N^2$.

In [60]:
m = random.randint(0, N)
assert 0 <= m < N

r = random.randint(0, N)
assert 0 < r < N

assert math.gcd(r, N) == 1

In [61]:
c = (pow(G, m, N**2) * pow(r, N, N**2)) % N ** 2

In [62]:
print("Ciphtertext Text: %s" % c)

Ciphtertext Text: 355319100241451033541678088643877898833979131980785565132253473627608284412518104382739852508828685789223898099078164031901272241045012325288580634271971446918647802183738986214517498660096487876843144962302019269137721221543155831541512107821655548732184219557695596498710278766050119198916396855273490088


### Decryption:

To decrypt a ciphertext $c \in Z_N$, let $L$ be a function defined over the set $\{u\in Z_{N^2}: u=1 \ mod \ N\}$ computed as $L(u) = \frac{u-1}{N}$. Then the decryption of $c$ is computed as

$$
\frac{L(c^{\lambda(N)}\ mod\ N^2)}{\mu}\ mod\ N
$$

In [63]:
(L(pow(c, Lam, N ** 2)) * M) % N

9396608797655379345128456032076009932115811181205074875432518483712562534239735487603048281378115238259546995302633433341476995534504412489865531607029354

In [64]:
m

9396608797655379345128456032076009932115811181205074875432518483712562534239735487603048281378115238259546995302633433341476995534504412489865531607029354

## Ref

* Wikipedia: Pillier Cryptosystem https://en.wikipedia.org/wiki/Paillier_cryptosystem


* Encryption Performance Improvementsof the Paillier Cryptosystem https://eprint.iacr.org/2015/864.pdf


* Stackoverflow - Paillier algorithm encryption https://stackoverflow.com/questions/29217630/paillier-algorithm-encryption